In [ ]:
from transformers import pipeline
ner = pipeline(
    "ner",
    model="dbmdz/bert-large-cased-finetuned-conll03-english",
    tokenizer="bert-large-cased",
    aggregation_strategy="first"
)
# testing
text = "company like Patagonia publicly supports environmental sustainability by promoting eco-friendly practices and advocating for conservation."
print(ner(text))

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps:0


[{'entity_group': 'ORG', 'score': 0.98126787, 'word': 'Patagonia', 'start': 13, 'end': 22}]


In [ ]:
import pandas as pd
from tqdm import tqdm
import numpy as np
link = 'documents.xlsx'
df = pd.ExcelFile(link).parse('general_labeled')

In [18]:
df.columns = df.iloc[0]
df = df[1:].reset_index(drop=True)

In [19]:
#df['recognition_value'] = df['recognition_value'].fillna('none').str.lower()

Unsurprisingly, making sentences into lower case worsen the results considerably - it is difficult for a model to recognise names

In [20]:
df['recognition_value'] = df['recognition_value'].fillna('none')

In [21]:
ner_output = []
for i in tqdm(range(800)):
    text = df['recognition_value'][i]
    ner_results = ner(text)
    ner_output.append(ner_results)

100%|██████████| 800/800 [01:22<00:00,  9.74it/s]


In [22]:
df['ner_output'] = ner_output

Descriptive Statistics on NER outputs:
    Progressive vs Conservative


In [23]:
def transform(row):
    if row != []:
        list_list_values = []
        for i in row:
            list_values = []
            for j in i.values():
                list_values.append(j)
            list_list_values.append(list_values)
        row = list_list_values
    else:
        row = []
    return row

In [24]:
trial = transform(df['ner_output'][1])

In [25]:
def dict_transform(row_transformed):
    df_nes_list = []
    for i in row_transformed:
        df_nes = {}
        df_nes[i[0]] = i[2]
        df_nes_list.append(df_nes)
    return df_nes_list

In [26]:
transformed = []
for i in tqdm(range(800)):
    row_transformed = transform(df['ner_output'][i])
    row_keyed = dict_transform(row_transformed)
    transformed.append(row_keyed)

100%|██████████| 800/800 [00:00<00:00, 32726.77it/s]


In [27]:
df['ner_output_first'] = transformed

In [28]:
df[['ner_output_first', 'recognition_value']].to_json('ner_output_first.json')

In [30]:
df[['ner_output_first', 'recognition_value']].to_excel('ner_output_first.xlsx')